# Working with mean and standard deviation
## Get the euclidean distance between the model cc cubic polynomial, and find acceptance probability. - Statistical metric for this. What is the probability that a point is a part of the normal distribution. 
### I model the a,b,c,d as a multimodel normal distrbution. So I have a list of these normal distributions, and then I get the probability that this particular data point is from this distribuition. 
### This is the theory for a *guassian mixture model*, except here I know the clusters from before.

#### We would look only at the starting feature - the first feature
#### We would look at all a,b,c,d firsty and then see for a,b,c as well.

##### Lets see what is the value for the results 

#### Testing the accuracy of model parameters

In [ ]:
vals, new_vals, cc_gaussian_params = train(var,s=1,e=10)

In [ ]:
def collect_test_data(var, s=11, e=20):
    cc_coeff_test = getCCcoeff(var,11,20)
    vals_test, new_vals_test = getCoeff(cc_coeff_test, "", p="n")
    return vals_test, new_vals_test

In [ ]:
vals_test, new_vals_test = collect_test_data(var)

In [ ]:
vals_test

In [ ]:
def get_test_accuracy(vals_test, cc_gaussian_param):
    acc_m = {}
    for cc in vals_test:
        if cc not in acc_m :
            acc_m[cc] = []
        data = vals_test[cc][1]
        for curr in data:
            p_dense = getPDensity(curr, cc_gaussian_params)
            acc_m[cc].append(p_dense)
    top = {}
    error = {}
    for cc in acc_m :
        top[cc] = {}
        ind = 0
        for item in acc_m[cc]:
            ccs = np.array(list(item.keys()))
            vals = np.array(list(item.values()))
            if min(vals) == 0:
                if max(vals) < 1 : 
                    #This is an arbitrary constant
                    if cc not in error :
                        error[cc] = []
                        error[cc].append(ind)
                        ind+=1
                        continue        
            new_vals = (vals-min(vals))/(max(vals)-min(vals))
            index_list =list(np.argsort(new_vals))
            index_list.reverse()
            cc_list = [ccs[i] for i in index_list]
            vals_list = [vals[i] for i in index_list]
            new_vals_list = [new_vals[i] for i in index_list]
            top[cc][ind] = {}
            top[cc][ind]['cc'] = cc_list[0:3]
            top[cc][ind]['vals'] = vals_list[0:3]
            top[cc][ind]['new_vals'] = new_vals_list[0:3]
            ind+=1
    return acc_m, top, error

In [ ]:
acc_m, top, error = get_test_accuracy(vals_test, cc_gaussian_params)

In [ ]:
def print_top_predictions(top):
    for cc in top :
        print()
        print(cc)
        print()
        for i in top[cc]:
            print(i)
            for d in top[cc][i]:
                print(top[cc][i][d])


In [ ]:
print_top_predictions(top)

In [ ]:
def getAccuracy(top):
    acc_data = {}
    for cc in top :
        acc_data[cc] = {}
        correct = 0
        incorrect = 0
        for item in top[cc]:
            curr = top[cc][item]
            for i in range(min(3,len(curr['new_vals']))):
                if curr['new_vals'][i] > 0.90 :
                    if curr['cc'][i] not in acc_data[cc]:
                        acc_data[cc][curr['cc'][i]] = 0
                    acc_data[cc][curr['cc'][i]] += 1
                    if curr['cc'][i] == cc :
                        correct += 1
                    else : 
                        incorrect += 1
        incorrect += 10 - len(top[cc])
        acc_data[cc]['correct'] = correct
        acc_data[cc]['incorrect'] = incorrect
        acc_data[cc]['accuracy'] = (correct*100)/(correct+incorrect)
    return acc_data

In [ ]:
acc_data = getAccuracy(top)

In [ ]:
def print_accuracy(acc_data):
    for cc in acc_data:
        new_dict = acc_data[cc]
        print()
        print(cc)
        print("Accuracy :", round(new_dict['accuracy'],2))

In [ ]:
# Run all once 
vals, new_vals, cc_gaussian_params = train(var,s=1,e=10)
vals_test, new_vals_test = collect_test_data(var,s=10,e=20)
acc_m, top, error = get_test_accuracy(vals_test, cc_gaussian_params)
acc_data = getAccuracy(top)

### Sample - 500 and Cubic

In [ ]:
print_accuracy(acc_data)

### Sample - 500 and Quadratic

In [ ]:
acc_m

In [ ]:
print_accuracy(acc_data)

## Trained on more data 

In [ ]:
acc_data

In [ ]:
# results = getRed(['cubic-1-0-50-1000-2'], p='y')
# error
for cc in error :
    print(cc)
    for i in range(1,11):
        print(i)
        print()
        file = cc + "-" + str(i) + '-0-50-1000-2'
        res = getRed([file], p='y')
        

## Training for model parameters

In [ ]:
def train(var,s=1,e=10):
    cc_coeff = getCCcoeff(var,s=s,e=e)
    vals, new_vals = getCoeff(cc_coeff, "", p="n")
    cc_gaussian_params = getGaussianParams(vals)
    return vals, new_vals, cc_gaussian_params

In [ ]:
def getGaussianParams(vals):
    cc_gaussian_params = {}
    for cc in vals :
        # Taking the first feature only
        data = vals[cc][1]
        n = 0
        cc_coeff_mean = np.mean(data,axis=0)
        coeff_var = np.cov(data, rowvar=0)
        iden = np.identity(len(coeff_var))
        cc_coeff_var = coeff_var * iden
        cc_gaussian_params[cc] = {
            'mean' : cc_coeff_mean,
            'covar' : cc_coeff_var
        }
    return cc_gaussian_params

In [ ]:
from scipy.stats import multivariate_normal as mvn
def getPDensity(curr, cc_gaussian_params):
    prob = {}
    for cc in cc_gaussian_params:
        mn = cc_gaussian_params[cc]['mean']
        covar = cc_gaussian_params[cc]['covar']
        curr_p = mvn.pdf(curr,mean=mn, cov=covar)
        prob[cc]=curr_p
    return prob

In [ ]:
def getCCcoeff(var,s=1,e=10,p="n"):
    cc_coeff = {}
    for v in var :
        files = [v + "-" + str(i) + "-0-50-1000-2" for i in range(s,e+1)]
        cc_mp, lin, quad, cub, errors = get_feature_degree(files,ss=524,p=p)
        coeff = getCC(files, cc_mp,p=p)
        cc_coeff[v] = coeff[v]
        #     getRed(files,p="y")
    return cc_coeff 

In [ ]:
get_feature_degree(['cubic-1-0-50-1000-2'],ss=524,p="n")

#### Below is testing data

In [ ]:
cc_gaussian_params = getGaussianParams(vals)
prob = getPDensity(vals['bic'][1][1], cc_gaussian_params)

In [ ]:
for cc in vals :
    data = vals[cc][1]
    cc_coeff_mean = np.mean(data,axis=0)
    cc_coeff_var = np.cov(data, rowvar=0)
    print(cc)
    print(cc_coeff_mean)
    print(cc_coeff_var)
    

# Getting a horizontal graph with ranges 

In [ ]:
gantt = {}
ytick = []
for cc in new_vals :
    ytick.append(cc)
    for ft in range(1,4):
        if ft not in gantt:
            gantt[ft] = {}
        if ft not in new_vals[cc]:
            coeff = {'a':[0],'b':[0],'c':[0],'d':[0]}
        else :
            coeff = new_vals[cc][ft]
        for cf in coeff : 
            if cf not in gantt[ft]:
                gantt[ft][cf] = []
            mn = mean(coeff[cf])
            std = pstdev(coeff[cf])
#             gantt[ft][cf].append((min(coeff[cf]), max(coeff[cf])-min(coeff[cf])))
            gantt[ft][cf].append((mn-std, 2*std))
#             gantt[ft][cf].append((0,mn))

In [ ]:
def make_gantt_chart(ax,x,ytick):
    ytick_points=[]
    new_ytick = []
    start = 0.3
    index_list =  [i[0] for i in sorted(enumerate(x), key=lambda x:x[1][0])]
    print(x)
    print(index_list)
    for ind in index_list:
        ax.broken_barh(xranges=[x[ind]], yrange=(start,0.5))
        new_ytick.append(ytick[ind])
        ytick_points.append(start)
        start+=1
    ax.set_yticks(ytick_points)
    ax.set_yticklabels(new_ytick)
    ax.set_title(coeff)
    plt.show()
#     ax.set_yticklabels(map(textwrap_func, ax.get_yticklabels()))

## Min and Range

In [ ]:
for index in gantt:
    print("index of feature ", index)
    for coeff in gantt[index]:
        fig, ax = plt.subplots(figsize=(10,10))
        make_gantt_chart(ax,gantt[index][coeff],ytick)

## Mean and Standard Deviation

In [ ]:
for index in gantt:
    print("index of feature ", index)
    for coeff in gantt[index]:
        fig, ax = plt.subplots(figsize=(10,10))
        make_gantt_chart(ax,gantt[index][coeff],ytick)

## Mean from 0

In [ ]:
for index in gantt:
    print("index of feature ", index)
    for coeff in gantt[index]:
        fig, ax = plt.subplots(figsize=(10,10))
        make_gantt_chart(ax,gantt[index][coeff],ytick)

In [ ]:
# CC
#   - 1 Index
#   - coeff - all the coeff

## Running the same thing on 10 experiments to understand the distribution of the coeff for particular algos

In [ ]:
var = ['cubic','reno']

In [ ]:
cc_coeff = getCCcoeff(var,s=11,e=19,p="y")

In [ ]:
cc_coeff

In [ ]:
vals = {}
for cc in cc_coeff:
    coeff = cc_coeff[cc]
    if cc not in vals :
        vals[cc] = {}
    for trace in coeff:
        i = 1
        for feature in trace:
            if i not in vals[cc]:
                vals[cc][i] = []
            vals[cc][i].append(feature)
            i+=1

In [ ]:
for cc in vals:
    items = vals[cc]
    for i in items :
        print(i)
        for co in items[i]:
            print(co)

In [ ]:
from statistics import median

In [ ]:
def getCoeff(cc_coeff, gcc="", p='y'):
    vals = {}
    degree = 0
    for cc in cc_coeff:
        coeff = cc_coeff[cc]
        if cc not in vals :
            vals[cc] = {}
        for trace in coeff:
            i = 1
            for feature in trace:
                if i not in vals[cc]:
                    vals[cc][i] = []
                degree = len(feature)
                vals[cc][i].append(feature)
                i+=1
    new_vals = {}
    coe = ['a','b','c','d']
    for cc in vals:
        new_vals[cc] = {}
        for i in vals[cc]:
            new_vals[cc][i] = {}
            c_val = {'a':[],'b':[],'c':[],'d':[]}
            for ft in vals[cc][i]:
                for x in range(degree):
                    c_val[coe[x]].append(ft[x])
            for x in range(degree):
                new_vals[cc][i][coe[x]] = c_val[coe[x]]
    if p == 'y':
        for cc in new_vals : 
            print(cc)
            if gcc != "" :
                if gcc != cc :
                    continue
            print(cc)
            for i in new_vals[cc] :
                print("Feature Index ", i)
                for c in coe :
                    x = new_vals[cc][i][c]
                    mn = mean(x)
                    std = pstdev(x)
                    plt.hist(x,bins=10)
                    plt.vlines(mn+std,0,3,color='r')
                    plt.vlines(mn-std,0,3,color='r')
                    plt.title(str(i) + " " + c)
                    plt.show()
                    plt.show()
#                 mn = mean(b)
#                 std = pstdev(b)
#                 plt.hist(b,bins=10)
#                 plt.vlines(mn+std,0,3,color='r')
#                 plt.vlines(mn-std,0,3,color='r')
#                 plt.title(str(i) + " b")
#                 plt.show()
#                 mn = mean(c)
#                 std = pstdev(c)
#                 plt.hist(c,bins=10)
#                 plt.vlines(mn+std,0,3,color='r')
#                 plt.vlines(mn-std,0,3,color='r')
#                 plt.title(str(i) + " c")
#                 plt.show()
#                 mn = mean(d)
#                 std = pstdev(d)
#                 plt.hist(d,bins=10)
#                 plt.vlines(mn+std,0,3,color='r')
#                 plt.vlines(mn-std,0,3,color='r')
#                 plt.title(str(i) + " d")
    return vals, new_vals

In [ ]:
vals, new_vals = getCoeff(cc_coeff, "", p="n")

In [ ]:
vals, new_vals = getCoeff(cc_coeff, "cubic")

In [ ]:
var = ["cubic", "reno"]

In [ ]:
var = "bic dctcp highspeed htcp illinois lp nv scalable vegas veno westwood yeah cubic reno"
#"cdg hybla"
var = var.split(" ")
print(var)

In [ ]:
from sklearn.metrics import mean_squared_error as mse
def get_degree(time,data, p="n", check=50):
    p = []
    mse_l = []
    time -= min(time)
    data -= min(data)
    data /= 1000
    p1 = np.polyfit(time, data, 1)
    p.append(p1)
    fit_l = np.polyval(p1, time)
    
    p2 = np.polyfit(time, data, 2)
    p.append(p2)
    fit_q = np.polyval(p2, time)
    
    p3 = np.polyfit(time, data, 3)
    p.append(p3)
    fit_c = np.polyval(p3, time)
    mse_l.append(mse(data, fit_l))
    mse_l.append(mse(data, fit_q))
    mse_l.append(mse(data, fit_c))
    if p =='y':
        print("1 ", p1, "MSE ", mse(data, fit_l))
        print("2 ", p2, "MSE ", mse(data, fit_q))
        print("3 ", p3, "MSE ", mse(data, fit_c))
        plt.plot(time, data)
        plt.plot(time, fit_l)
        plt.plot(time, fit_q)
        plt.plot(time, fit_c)
        plt.show()
    check = check
    if mse_l[1] < 0.50*mse_l[0] :
        if mse_l[2] < 0.50*mse_l[1] :
            degree = 3
        else :
            degree = 2
    else :
        degree = 1
#         if 0.5*(mse_l[0]-mse_l[1]) > mse_l[1]-mse_la :
#         a[2] :
#             degree = 1
#         else:
#             degree = 2
#     else:
#         degree = 0
#     if mse_l[2#     for i in 1](0,len(mse_l)-1):
#         diff = abs(mse_l[len(mse_l)-1] - mse_l[i])
#         perc = (diff/mse_l[len(mse_l)-1])*100
#         if perc < check :
#             return i+1,p[i], mse_l
#     return degree, p[degree-1], mse_l
#     return 2, p[1], mse_l
    return 3,p[2], mse_l
# get_degree(time, data)
from statistics import mean 
def get_feature_degree(files,ss=125,p='n'):
    print(ss)
    results = getRed(files,ss,p=p)
    errors = []
    mp = {}
    lin = {}
    quad = {}
    cub = {}
    cc_mp = {}
    for item in results :
        for ele in list(item.keys()):
            name_list = ele.split("_")
            cc = name_list[0]
            name = name_list[0] + name_list[-1]
            if "data" in ele :
                curr_data = np.array(item[ele])
            if "time" in ele :
                curr_time = np.array(item[ele])
        degree, coeff, error_item = get_degree(curr_time, curr_data, check=25)
        mp[name] = {'d':degree, 'coeff':coeff, 'error':error_item, 'data':curr_data, 'time':curr_time}
        if cc not in cc_mp :
            cc_mp[cc] = []
        cc_mp[cc].append(mp[name])
        if degree == 1 :
            lin[name] = mp[name]
        if degree == 2:
            quad[name] = mp[name]
        if degree == 3:
            cub[name] = mp[name]
        error_item.append(name)
        error_item.append(degree)
        errors.append(error_item)
    return cc_mp, lin, quad, cub, errors

def getCC(files,cc_mp, s=100, p="n"):
    # experiment change start
    cc_coeff = {}
    for file in files :
#         file = v + "-0-50-1000-2"
        curr_file = [file]
        cc = file.split("-")[0]
        version = file.split("-")[1]
        v = cc+"-"+version
        if cc not in cc_coeff:
            cc_coeff[cc] = []
    # experiment change end
        features, time, data = get_new_grad(curr_file, 'n')
        count = 0
        temp = []
        n = math.ceil(float(len(cc_mp[v]))/3)
        for item in cc_mp[v]:
            time = item['time']
            data = item['data']
#             time -= min(time)
#             data -= min(data)
#             data /= 1000
            deg = item['d']
            temp.append(item['coeff'])    
            xlim = 0
            ylim = 0
            for i in [0.5,1,2,4,8,16]:
                if time[-1]<i:
                    xlim = i
                    break
            for i in [0.5,1,2,4,8,16]:
                if data[-1]<i:
                    ylim = i
                    break
#             if data[-1] < 1 :
            lim = max(xlim, ylim)
            names = ['Linear', "Quadratic", "Cubic"]
            count+=1
            if p == 'y':
                plt.plot(time,data)
                plt.plot(time, np.polyval(item['coeff'],time))
                plt.xlim(0,lim)
                plt.ylim(0,lim)
                plt.title(str(count)+" " + names[deg-1])
                plt.show()
                plt.figure().set_figwidth(4)
                plt.figure().set_figheight(2)
                plt.bar([i for i in range(1,deg+2)], item['coeff'])
                plt.show()
                plt.figure().set_figwidth(4)
                plt.figure().set_figheight(2)
                plt.bar([1,2,3], item['error'][0:3], tick_label=['lin', 'quad', 'cubic'])
                plt.show()
        cc_coeff[cc].append(temp)
    return cc_coeff 

In [ ]:
import bisect
def lower_bound(arr, target):
    index = bisect.bisect_left(arr, target)
    return index

def sample_data_time(time, data, s, m):
    curr_time, curr_data = adjust(time, data, 0.10)
    tp = curr_time[len(curr_time)-1] - curr_time[0]
    step = tp/m
    samp_time = [curr_time[0] + i*step for i in range(m)]
    x = np.random.uniform(0,math.pi,s)
    tr_x = np.cos(x)
    tr_x += 1
    tr_x *= (m-1)/2
    ind = [int(round(e, 0)) for e in tr_x]
    sort_ind = sorted(ind)
    tr_time = [samp_time[i] for i in sort_ind]

    new_time = []
    new_data = []
    for t in tr_time :
        i = lower_bound(curr_time, t)
        temp_t = 0
        temp_d = 0
        if round(t,6) == round(curr_time[i],6):
            temp_t = curr_time[i]
            temp_d = curr_data[i]
        else : 
            if i == 0 :
                temp_t = curr_time[i]
                temp_d = curr_data[i]
            elif i == len(curr_time)-1 :
                temp_t = curr_time[i]
                temp_d = curr_data[i]
            else :
                temp_t = (curr_time[i-1] + curr_time[i])/2
                temp_d = (curr_data[i-1] + curr_data[i])/2
        new_time.append(temp_t)
        new_data.append(temp_d)
#     return curr_time, curr_data
    return new_time, new_data

def adjust(time, data, m=0.25):
    control = 100000;
    rtt=0.1
    for t in range(len(time)) :
        if time[t]-time[0] > 5*rtt:
            control = t; 
            break
    l = min(t,int(m*len(time)))
    e = int(len(time)-l)
    start = max(data.index(min(data[0:l])),data.index(max(data[0:l])))
    end = min(data.index(max(data[e:])), data.index(min(data[e:])))
    new_time = time[start:end+1]
    new_data = data[start:end+1]
    return new_time, new_data

# Taking 100 as the threshold is fine 
# Now we have to see how the graphs look if we use it. 
# var = ["reno", "cubic", "bbr"]
def getRed(files,ss=125,p="y"):
    results = []
    rtt = 0.1
    thresh = 5*rtt
    # experiment change start
    for file in files :
#         file = v + "-0-50-1000-2"
    # experiment change end    
        f_split = file.split("-")
        v = f_split[0] + "-" + f_split[1]
        curr_file = [file]
        features, time, data = get_new_grad(curr_file, p)
        count = 0
        for ft in features : 
            if ft[2]==0:
                time_period = time[ft[1]] - time[ft[0]]
                if time_period >= thresh:
#                     print(time[ft[0]], time[ft[1]])
                    curr_time = time[ft[0]:ft[1]+1]
                    curr_data = data[ft[0]:ft[1]+1]
                    tr_time, tr_data = sample_data_time(curr_time, curr_data, ss, 1000)
                    tr_time_pd = pd.DataFrame(tr_time)
                    tr_data_pd = pd.DataFrame(tr_data)
                    tr_time = list(tr_time_pd.rolling(25, center=True).mean().dropna()[0])
                    tr_data = list(tr_data_pd.rolling(25, center=True).mean().dropna()[0])
#                     print(tr_time)
#                     print(tr_time_t)
#                     print(tr_data)
#                     print(tr_data_t)
                    print("Data and Time Length ", len(tr_data), len(tr_time))
                    if p == "y" :
                        plt.plot(curr_time, curr_data, c='b', alpha = 0.5, lw = 5)
                        plt.plot(tr_time, tr_data, c='r', alpha = 1)
                        plt.scatter(tr_time, tr_data, c='k')
        #                 plt.scatter(tr_time, tr_data, c='r', s=10)
                        plt.title(v)
                        plt.show()
#                     print("Data Difference : " + str(tr_data[-1]-tr_data[0]))
#                     print("Time Difference : " + str(tr_time[-1]-tr_time[0]))
                    results.append({v+"_"+"data"+"_"+str(count):tr_data, v+"_"+"time"+"_"+str(count):tr_time})
                    count+=1
    return results
# results = getRed(var)

In [ ]:
results

In [ ]:
for i in range(1,31):
#     results = getRed(['reno-' + str(i) + '-0-50-1000-2'],ss=524, p='y')
    plot_one_bt(['cubic-' + str(i) + '-0-50-1000-2'], p= "y")
#     get_new_grad(['reno-' + str(i) + '-0-50-1000-2'], p= "y")

In [ ]:
import matplotlib.pyplot as plt
from scipy.fft import rfft, rfftfreq
from scipy.fft import irfft
import numpy as np
import statistics
from statistics import mean, pstdev
import pandas as pd

def smoothen(time, data, rtt):
    # Smoothening 
    left = 0
    right = 0
    run_sum = 0
    avg_data = []
    new_time = []
    roll_time = time
    roll_data = data
#     while(time[right]-time[left] < float(rtt)/2):
#         avg_data.append(data[right])
#         new_time.append(time[right])
#         right+=1
#     roll_data = pd.DataFrame(data)
#     roll_time = pd.DataFrame(time)
#     roll_data =roll_data.rolling(10,center=True).mean().dropna()
#     roll_time =roll_time.rolling(10,center=True).mean().dropna()
#     roll_data = list(roll_data[0])
#     roll_time = list(roll_time[0])
    while right < len(roll_time):
        while(right < len(roll_time) and (roll_time[right]-roll_time[left] < 2*rtt)):
            run_sum+=roll_data[right]
            right+=1
        new_time.append(float(roll_time[right-1]+roll_time[left])/2)
        avg_data.append(float(run_sum)/(right-left))
        run_sum-=roll_data[left]
        left+=1
    #experiment change start
#     avg_data = pd.DataFrame(avg_data)
#     new_time = pd.DataFrame(new_time)
#     avg_data =avg_data.rolling(50,center=True).mean().dropna()
#     new_time =new_time.rolling(50, center=True).mean().dropna()
#     avg_data = list(avg_data[0])
#     new_time = list(new_time[0])
#     print(avg_data)
#     print(new_time)
    #experiment end
#     right-=1
#     prev = right
#     while(time[right] - time[prev] < float(rtt)/2):
#         prev-=1
#     prev+=1
#     for i in range(prev, right+1):
#         avg_data.append(data[i])
#         new_time.append(time[i])
#     print(len(data), len(avg_data), len(new_time))
#     avg_data = n_data.rolling(window=60, center=True).mean()
#     new_time = time
#     for i in range(30):
#         avg_data[0][i] = data[i]
#     for i in range(-1, -30, -1):
#         avg_data[0][len(data) + i] = data[len(data) + i]
#     ans_data = avg_data[0]
#     Smoothen function works well.
    return new_time, avg_data
#     return roll_time, roll_data

def get_fft(data):
    n = len(data)
    data_step = 0.002
    yf = rfft(data)
    xf = rfftfreq(n,data_step)
    return yf,xf

def get_fft_smoothening(ax,f,norm,rtt,p):
    rtt=rtt
    data, time = get_window(f, "n")

    yf, xf = get_fft(data)
    abs_yf = np.abs(yf)
#     print(len(abs_yf), len(xf))
    sort_abs_yf = sorted(abs_yf)
    
#     plt.title("FFT Decomposition")
#     plt.plot(xf, sort_abs_yf, color='b', alpha=0.5, lw=1)
#     plt.scatter(xf, sort_abs_yf, color='r', s=2)
#     plt.show()

    thresh  = (1/rtt)
#     print("RTT", rtt, "Freq ", thresh)
    thresh_ind = 0
    for i in range(len(xf)) :
        freq = xf[i]
        if(freq > thresh):
            thresh_ind = i
            break
#     print("Index ", thresh_ind)
#     print(xf[0:thresh_ind])
#     plt.title("Cleaned Frequencies")
#     plt.plot(xf[0:thresh_ind],np.abs(yf[0:thresh_ind]))
#     plt.show()

    yf_clean = yf
    yf_clean[thresh_ind+1:] = 0
    new_f_clean = irfft(yf_clean)
#     plt.title("Frequencies smaller than " + str(thresh))
#     plt.plot(time, data, color="b", alpha=0.5, label='original signal')
    start_len = len(time) - len(new_f_clean)

    plot_data = new_f_clean
    
    if norm == 1:
        max_data = max(new_f_clean)
        min_data = min(new_f_clean)
        plot_data = (new_f_clean - min_data)/(max_data - min_data)
    if p:
        ax.plot(time[start_len:], plot_data, 'k', label='FFT smoothening', linewidth=1.5)
#     plt.legend()
#     plt.show()
    plot_time = time[start_len:] 
    return plot_time, plot_data

def get_grad(time, data):
    new_data = []
#     new_data.append(0)
    for i in range(1, len(data)):
        # experiment change start 
        diff = time[i]-time[i-1]
        
        diff = max(0.000001,diff)
        # experiment end
#         diff=1
        new_data.append((data[i]-data[i-1])/(diff))
    new_time = time[1:]
    plot_data = np.array(new_data)
    max_data = max(new_data)
    min_data = min(new_data)
    new_data = np.array(new_data)
    plot_data = (new_data - min_data)/(max_data-min_data)
    return new_time, plot_data
def plot_d(ax, time, data, c, l):
    ax.plot(time, data, color=c, lw=2, label = l)

def smooth_grad(grad_time, grad_data):
    dt = pd.DataFrame(grad_data)
    tm = pd.DataFrame(grad_time)
    dt = list(dt.rolling(50, center=True).mean().dropna()[0])
    tm = list(tm.rolling(50, center=True).mean().dropna()[0])
    return tm, dt

def plot_one_bt(files, p):
    for f in files:
        fs = f.split("-")
        # experiment change start 
        pre = int(fs[2])
        post = int(fs[3])
        # experiment change end
        rtt = float(((pre+post)*2))/1000
#         print(rtt)
        ax = 0
        if p == 'y':
            fig, ax = plt.subplots(1,1, figsize=(15,8))
        
        time, data = get_fft_smoothening(ax,f,0,rtt,0)
#         plot_d(ax,time,data,"b","FFT Smoothened" )
#         print(len(time), len(data))
        time, data = smoothen(time, data, rtt)
#         print(len(time), len(data))
        if p == 'y':
#             plot_d(ax, time, data, "r", "Smoothened")
            plot_data = np.array(data)
            plot_data -= min(data)
            plot_data /= (max(data)-min(data))
            plot_d(ax, time, plot_data, "g", "Smoothened")
#         fd_time, fd_data = get_fd(time, data)
#         plot_d(ax, time, data, "g")
        grad_time, grad_data = get_grad(time, data)
#         smooth_grad_time, smooth_grad_data = smooth_grad(grad_time, grad_data)
        if p == 'y':
#             grad_plot_data += min(data)
            plot_d(ax, grad_time, grad_data, "r", "Gradient")
#             plot_d(ax, smooth_grad_time, smooth_grad_data, "b", "Smooth Gradient")
#             start = math.floor(0.05*len(grad_time))
            devi = statistics.pstdev(grad_data)
            mean = statistics.mean(grad_data)
            m = 0.60
            plt.fill_between(grad_time, [mean + devi*m for i in range(len(grad_time))], [mean - devi*m for i in range(len(grad_time))], color='y')
#         grad_step_time, grad_step_data = get_step_grad(time, data, rtt)
#         plot_d(ax, grad_step_time, grad_step_data, "b", "Step Gradient")
#         ax.title.set_text(f)
            ax.legend()
            plt.show()
    return time, data, grad_time, grad_data, rtt
#     return time, data 
def get_time_features_bt(curr_file, p):
    main_time, main_data, grad_time, grad_data, rtt= plot_one_bt(curr_file, p)
    
    data = grad_data
    time = grad_time
#     start = math.floor(0.05*len(grad_time))
    mean = data.mean()
#     print("Mean", mean)
    sd = statistics.pstdev(data)
#     print("Std Dev", sd)
    data = abs(data - mean)
    features = []
    tp = 0
    ch_left = 0
    ch_right = 0
    max_win = 0
    min_win = 0
    m = 0.60
    while time[ch_right] - time[ch_left] < rtt :
        max_win = max(max_win, data[ch_right])
        ch_right+=1
    if max_win > m*sd :
        tp = 1

    max_win = 0
    min_win = 0

    ft_left = 0
    left = 0
    right = 0

    while ft_left < len(time) and left < len(time):
        while right < len(time) and (time[right] - time[left]) <= rtt:
            max_win = max(max_win, data[right])
            right+=1
        if tp == 0:
            if (max_win > m*sd):
                features.append([ft_left, left-1, tp])
                ft_left = left
                tp = 1
        if tp == 1:
            if (max_win < m*sd):
                features.append([ft_left, left-1, tp])
                ft_left = left
                tp = 0
    #     print(left, right, max_win, mean + m*sd)
        max_win = 0
        left = right 

    features.append([ft_left, left-1, tp])
    return features, main_time, main_data, grad_time, grad_data, rtt
def get_amp_grad(features, grad_time, grad_data, rtt):
#     mn = grad_data.mean()
#     grad_data -= mn
#     for ft in features :
# #         This idea should be more refined mathematically
#         if ft[2]==1:
#             grad_data[ft[0]:ft[1]+1]*=2
#         if ft[2]==0:
#             grad_data[ft[0]:ft[1]+1]/=2
#     grad_data += mn
#     data = grad_data
#     time = grad_time
#     mean = data.mean()
#     print("Mean", mean)
#     sd = statistics.pstdev(data)
#     print("Std Dev", sd)
#     data = abs(data - mean)
#     features = []
#     tp = 0
#     ch_left = 0
#     ch_right = 0
#     max_win = 0
#     min_win = 0
#     m = 2
#     while time[ch_right] - time[ch_left] < rtt :
#         max_win = max(max_win, data[ch_right])
#         ch_right+=1
#     if max_win > m*sd :
#         tp = 1

#     max_win = 0
#     min_win = 0

#     ft_left = 0
#     left = 0
#     right = 0

#     while ft_left < len(time) and left < len(time):
#         while right < len(time) and (time[right] - time[left]) <= rtt:
#             max_win = max(max_win, data[right])
#             right+=1
#         if tp == 0:
#             if (max_win > m*sd):
#                 features.append([ft_left, left-1, tp])
#                 ft_left = left-1
#                 tp = 1
#         if tp == 1:
#             if (max_win < m*sd):
#                 features.append([ft_left, left-1, tp])
#                 ft_left = left-1
#                 tp = 0
#     #     print(left, right, max_win, mean + m*sd)
#         max_win = 0
#         left = right
        
#     features.append([ft_left, left-1, tp])
    pos = 0
    for ft in features :
        if pos > 0 and pos < len(features)-1 :
            if(round(grad_time[ft[1]] - grad_time[ft[0]],3) <= 3*rtt):
                if features[pos-1][2] == features[pos+1][2]:
                    ft[2] = features[pos-1][2]
#                     model = LinearRegression()
#                     curr_range = list(range(features[pos-1][0],features[pos-1][1]+1))
#                     X = np.reshape(curr_range, (len(curr_range), 1))
#                     model.fit(X, data[features[pos-1][0]:features[pos-1][1]+1])
#                     left = statistics.mean(model.predict(X))
#                     print("Left",left)
#                     curr_range = list(range(features[pos+1][0],features[pos+1][1]+1))
#                     X = np.reshape(curr_range, (len(curr_range), 1))
#                     model.fit(X, data[features[pos+1][0]:features[pos+1][1]+1])
#                     right = statistics.mean(model.predict(X))
#                     print("Right", right)
#                     if left*right < 0 and ft[2] == 0:
#                         ft[2] = 1
#                     if left*right > 0 and ft[2] == 1:
#                         ft[2] = 0            
        pos+=1
    new_ft = []
    start_type = features[0][2]
    start = features[0][0]
    end = 0
    for i in range(len(features)) :
        ft = features[i]
        curr_type = ft[2]
        if curr_type != start_type :
            end = ft[0]
            new_ft.append([start, end, start_type])
            start = end
            start_type = ft[2]
        if i == len(features)-1:
            new_ft.append([start, ft[1], start_type])
        
    return new_ft, grad_time, grad_data
#     return new_ft, grad_time, grad_data
def get_features(features, main_time, grad_time):
    left = 0
    right = 0
    result_ft = []
    for ft in features:
#         print("Next Stop", grad_time[ft[1]])
        while(right < len(main_time) and main_time[right] < grad_time[ft[1]]):
#             print(main_time[right])
            right+=1
        result_ft.append((left, right-1, ft[2]))
        left = right
    if left < right :
        result_ft.append((left, right-1, ft[2]))
    
    return result_ft
color = ['r', 'b']
def get_new_grad(curr_file, p):
    features, main_time, main_data, grad_time, grad_data, rtt = get_time_features_bt(curr_file, p)
    features, grad_time, grad_data = get_amp_grad(features, grad_time, grad_data, rtt)
    main_features = get_features(features, main_time, grad_time)
    if p == 'y' :
        fig, ax = plt.subplots(1,1, figsize=(15,8))
#         plot_d(ax, grad_time, grad_data, "y", "Smoothened Gradient")
        for ft in features : 
    #         print(main_time[ft[1]]-main_time[ft[0]], ft[2])
            ax.plot(main_time[ft[0]:ft[1]+1], main_data[ft[0]:ft[1]+1], color = color[ft[2]])
        plt.show()
    return features, main_time, main_data


In [ ]:
get_new_grad(['cubic-1-0-50-1000-1-aws'], p="y")

In [ ]:
time, data, grad_time, grad_data, rtt = plot_one_bt(['cubic-1-0-50-1000-2'], p="y")

In [ ]:
time, data, grad_time, grad_data, rtt = plot_one_bt(['cubic-1-0-50-1000-2'], p="y")

In [ ]:
time, data, grad_time, grad_data, rtt = plot_one_bt(['cubic-1-0-50-1000-2'], p="y")

In [ ]:
time, data, grad_time, grad_data, rtt = plot_one_bt(['bbr-1-0-50-1000-2-aws'], p="y")

In [ ]:
import csv
import matplotlib.pyplot as plt
import sys
import math

SHOW=True
MULTI_GRAPH=False
SMOOTHENING=False
ONLY_STATS=False
s_factor=0.9

'''
TODO: 
o Add functionality where you only plot flows that send more than x bytes of data
o Sort stats and graphs by flow size
o Organize plots by flow size (larger flows have larger graphs)
o Custom smoothening function
'''

fields=["time", "frame_time_rel", "tcp_time_rel", "frame_num", "frame_len", "ip_src", "src_port", "ip_dest", "dest_port", "tcp_len", "seq", "ack"]

class pkt:
    contents=[]
    def __init__(self, fields) -> None:
        self.contents=[]
        for f in fields:
            self.contents.append(f)

    def get(self, field):
        return self.contents[fields.index(field)]
        

def process_flows(cc, dir):
    with open(dir+cc+"-tcp.csv") as csv_file:
        print("Reading "+dir+cc+"-tcp.csv...")
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        total_bytes=0
        '''
        Flow tracking:
        o Identify all packets that are either sourced from or headed to 100.64.0.2
        o Group different flows by client's port
        '''
        flows={}
        data_sent=0
        port_set = set()
        for row in csv_reader:
            packet=pkt(row)
            validPkt=False
            if line_count==0:
                # reject the header
                line_count+=1
                continue
            if data_sent == 0 : 
                if "100.64.0." in packet.get("ip_src"):
                    num = int(packet.get("ip_src")[-1])
                    if num%2==0:
                        data_sent=1
                        host_port=packet.get("ip_src")
                if "100.64.0." in packet.get("ip_dest"):
                    num = int(packet.get("ip_dest")[-1])
                    if num%2==0:
                        data_sent=1
                        host_port=packet.get("ip_dest")
                if data_sent == 0:
                    continue
                    
            if packet.get("ip_src")==host_port and packet.get("frame_time_rel")!='' and packet.get("ack")!='': 
                # we care about this ACK packet
                validPkt=True
                port=packet.get("src_port")
                if port not in flows:
                    flows[port]={"serverip":packet.get("ip_dest"), "serverport":packet.get("dest_port"), "act_times":[],"times":[], "windows":[], "cwnd":[], "bif":0, "last_ack":0, "last_seq":0, "pif":0}
                flows[port]["act_times"].append(packet.get("time"))
                flows[port]["times"].append(float(packet.get("frame_time_rel")))
                flows[port]["bif"]-=max(0,(int(packet.get("ack"))-int(flows[port]["last_ack"])))
                flows[port]["last_ack"]=max(int(flows[port]["last_ack"]),int(packet.get("ack")))
                #flows[port]["windows"].append(int(flows[port]["bif"]))
                flows[port]["pif"]-=1
                flows[port]["cwnd"].append(flows[port]["pif"])
            elif packet.get("ip_dest")==host_port and packet.get("frame_time_rel")!='' and packet.get("seq")!='':
                #we care about this Data packet
                validPkt=True
                port=packet.get("dest_port")
                if port not in flows:
                    flows[port]={"serverip":packet.get("ip_src"), "serverport":packet.get("src_port"),"act_times":[], "times":[], "windows":[], "cwnd":[], "bif":0, "last_ack":0, "last_seq":0, "pif":0}
                flows[port]["times"].append(float(packet.get("frame_time_rel")))
                flows[port]["act_times"].append(packet.get("time"))
                flows[port]["bif"]+=max(0,(int(packet.get("seq"))-int(flows[port]["last_seq"])))
                flows[port]["last_seq"]=max(flows[port]["last_seq"],int(packet.get("seq")))
                flows[port]["pif"]+=1
                flows[port]["cwnd"].append(flows[port]["pif"])
            if SMOOTHENING and validPkt and len(flows[port]["windows"])>2:
                flows[port]["windows"].append(int((s_factor*flows[port]["windows"][-1])+((1-s_factor)*flows[port]["bif"])))
            elif validPkt:
                flows[port]["windows"].append(int(flows[port]["bif"]))
            line_count+=1
            total_bytes+=int(packet.get("frame_len"))
            #print(line_count, total_bytes)
            
#         print("total bytes processed:", total_bytes/1000, "KBytes for", cc, "(unlimited)")
    return flows

def custom_smooth_function():
    pass

def get_flow_stats(flows):
    num=len(flows.keys())
    print("FLOW STATISTICS: \nNumber of flows: ", num)
    print("------------------------------------------------------------------------------")
    print('%6s'%"port", '%15s'%"SrcIP", '%8s'%"SrcPort",  '%8s'%"duration",  '%8s'%"start",  '%8s'%"end", '%8s'%"Sent (B)", '%8s'%"Recv (B)",)
    for k in flows.keys():
        print('%6s'%k, '%15s'%flows[k]["serverip"], '%8s'%flows[k]["serverport"], '%8s'%str('%.2f'%(flows[k]["times"][-1]-flows[k]["times"][0])), '%8s'%str('%.2f'%flows[k]["times"][0]), '%8s'%str('%.2f'%flows[k]["times"][-1]), '%8s'%flows[k]["last_seq"], '%8s'%flows[k]["last_ack"])
        #print("    * Flow "+str(k)+": ", flows[k]["last_ack"], " ", flows[k]["last_seq"], " bytes transfered.")
    return num

def run(files):
    flows = {}
    for f in files:
        algo_cc=f
        #Get the data for all the flows
        print("==============================================================================")
        print("opening trace ../measurements/"+algo_cc+".csv...")
        flows = process_flows(algo_cc, "./Nebby/measurements/")
        #decide on final graph layout
        num = get_flow_stats(flows)

        if ONLY_STATS:
            sys.exit()

        if num==1:
            MULTI_GRAPH=False
        #grid size
        if MULTI_GRAPH:
            size=(0,0)
            grids={1:(2,2), 2:(2,2), 4:(2,2), 6:(2,3), 9:(3,3), 12:(3,4), 15:(3,5), 16:(4,4), 20:(5,4), 24:(6,4), 30:(6,5), 36:(6,6), 40:(8,5), 42:(8,7), 49:(7,7)}
            g=num
            while g<=49 and g not in grids:
                g+=1
            if g in grids.keys():
                size=grids[g]
            else:
                size=grids[49]  
            fig, axs = plt.subplots(size[0], size[1])
            for i in range(size[0]):
                for j in range(size[1]):
                    #axs[i][j].legend(loc="lower right")
                    if i==size[0]-1:
                        axs[i][j].set_xlabel("Time (s)")
                    if j==0:
                        axs[i][j].set_ylabel("Bytes in flight")
        else:
            plt.xlabel("Time (s)")
            plt.ylabel("Bytes in flight")
        counter=0
        for port in flows.keys():
            if MULTI_GRAPH:  
                axs[counter%size[0]][(counter//size[0])%size[1]].scatter(flows[port]["times"], flows[port]["windows"], color="#858585")
                axs[counter%size[0]][(counter//size[0])%size[1]].plot(flows[port]["times"], flows[port]["windows"], label=str(port), linestyle="solid")
            else:
                plt.plot(flows[port]["times"], flows[port]["windows"], label=str(port), linestyle="solid")
                plt.scatter(flows[port]["times"], flows[port]["windows"], color="#858585")
            counter+=1
        if MULTI_GRAPH:
            counter=0
            for port in flows.keys():
                axs[counter%size[0]][(counter//size[0])%size[1]].legend()
                counter+=1
        else:
            plt.legend()
        if MULTI_GRAPH:
            fig.set_size_inches(16, 12)
        if SHOW:
            plt.show()
        else:
            plt.savefig("../logs/results/"+algo_cc+".png", dpi=600, bbox_inches='tight', pad_inches=0)
    return flows

def get_window(f,p):
    algo_cc = f
    flows = process_flows(algo_cc, "./Nebby/measurements/")
    data = []
    time = []
    print(flows.keys())
    for port in flows.keys():
        if flows[port]["last_ack"] == 0 :
            continue
        else :
            print(port)
            data = flows[port]['windows']
            time = flows[port]['times']
            print(data)
            print(time)
            if p == "p":
                for i in range(len(flows[port]['windows'])):
                    print("{0:10}{1:10}".format(flows[port]['times'][i], flows[port]['windows'][i]))
        time_index = len(time)-1
        for index in range(len(flows[port]['times'])-1):
            if flows[port]['times'][index+1] - flows[port]['times'][index] > 1 :
                time_index = index
        time_last = flows[port]['times'][time_index]
        data = data[:time_index+1]
        time = time[:time_index+1]
    return data, time